# Week 13: Example Sentences Classification
In the last two assignment, we use deep learning method to do classification task. We can get good result by deep learning, but we can hardly explain the classification result(i.e., we don't know why the phrase is classified as "good" phrase). Hence, in this assignment, we want you do classification again, but do it by traditional machine learning method. In this way, you can know *why* more. We want you implement this on example sentences(e.g., "This song has a brilliant piano intro." for word "brilliant").

[Here](https://drive.google.com/drive/folders/1ij20ecLlI1Zh5CdMAa91SXQzmpKfdBdJ?usp=sharing) are two files needed for this task: **train.jsonl** & **test.jsonl**. In these files, each line records one example sentence with its label. There're two types of labels: ***positive*** means it's *good* example sentence;  ***negative*** means it's *bad* example sentence.

## Read Data
We use dataframe to keep data in this assignment.

In [1]:
import json
import pandas as pd

In [2]:
def read_data_to_df(path):
    labels, sentences = [], []
    with open(path, 'r') as f:
        for line in f.readlines():
            line = json.loads(line)
            sentences.append(line['text']), labels.append(line['label'])
    return pd.DataFrame({'sent':sentences, 'label':labels})  

In [3]:
train = read_data_to_df('data/train.jsonl')
print(train.head())
test = read_data_to_df('data/test.jsonl')
print(test.head())

                                                sent     label
0         My children threw a birthday party for me.  positive
1  Marketing on social networking sites is a fast...  positive
2                You pillock, look what you've done!  positive
3      He scored three touchdowns early in the game.  positive
4          His abrupt manner makes me uncomfortable.  positive
                                                sent     label
0  I've just spotted Mark - he's over there, near...  positive
1  After repricing, the bonds yield a much higher...  positive
2             I admire her staunchness and fidelity.  positive
3  The party's leader is in favour of the treaty ...  positive
4  About 20 companies are working on treatments f...  positive


## Extract Features
Traditional machine learning need data scientists to observe data and find out some useful information. 

Here is an example:

In [4]:
import re
# for positive
spec, count = 0, 0
for i in train[train['label'] == "positive"]["sent"]:
    if len(re.sub('[\w\s]','',i)) > 2: spec+=1
    count+=1
print(spec/count)

0.1658252289596287


In [5]:
spec, count = 0, 0
for i in train[train['label'] == "negative"]["sent"]:
    if len(re.sub('[\w\s]', '', i)) > 2: spec+=1
    count+=1
print(spec/count)

0.5131938403308762


After some experiment, we found 38% of bad example sentences have more than 3 punctuations. In contrast, only 5.8% of good example sentences have more than 3 punctuations. Hence, it seems that it is a nice feature to distinguish good and bad example sentences, so we add this feature.

In [6]:
train["more than n punc"] = [len(re.sub('[\w\s]','',i)) for i in train["sent"]]
test["more than n punc"] = [len(re.sub('[\w\s]','',i)) for i in test["sent"]]

In [7]:
train.head()

,sent,label,more than n punc
0,My children threw a birthday party for me.,positive,1
1,Marketing on social networking sites is a fast...,positive,2
2,"You pillock, look what you've done!",positive,3
3,He scored three touchdowns early in the game.,positive,1
4,His abrupt manner makes me uncomfortable.,positive,1


<font color="red">**[ TODO ]**</font> Please observe the data and extract at least three features and add them to the dataframe.

In [8]:
# Adding more than n words 1
n = 18
train["more than n words"] = [0 if len(re.findall(r'\w+', i)) >= n else 1 for i in train["sent"]]
test["more than n words"] = [0 if len(re.findall(r'\w+', i)) >= n else 1 for i in test["sent"]]
# Adding more than n capital 2
n = 1
train["more than n capital"] = [0 if sum([1 if i.istitle() else 0 for i in sen.split()]) > n else 1 for sen in train["sent"]]
test["more than n capital"] = [0 if sum([1 if i.istitle() else 0 for i in sen.split()]) > n else 1 for sen in test["sent"]]
# Adding more than n common 3
n = 0
train["more than n common"] = [0 if len(re.findall(r'\,', i)) > n else 1 for i in train["sent"]]
test["more than n common"] = [0 if len(re.findall(r'\,', i)) > n else 1 for i in test["sent"]]
# Adding more than n period 4
n = 2
train["more than n period"] = [0 if len(re.findall(r'\.', i)) > n else 1 for i in train["sent"]]
test["more than n period"] = [0 if len(re.findall(r'\.', i)) > n else 1 for i in test["sent"]]
# Adding more than n equal 5
n = 0
train["more than n equal"] = [0 if len(re.findall(r'\=', i)) > n else 1 for i in train["sent"]]
test["more than n equal"] = [0 if len(re.findall(r'\=', i)) > n else 1 for i in test["sent"]]
# Adding more than n ( 6
n = 0
train["more than n ("] = [0 if len(re.findall(r'\(', i)) > n else 1 for i in train["sent"]]
test["more than n ("] = [0 if len(re.findall(r'\(', i)) > n else 1 for i in test["sent"]]
# Adding more than n quotation 7
train["more than n quotation"] = [len(re.findall(r'\"', i)) for i in train["sent"]]
test["more than n quotation"] = [len(re.findall(r'\"', i)) for i in test["sent"]]
# Adding more than n space 8
n = 2
train["more than n space"] = [0 if len(re.findall(r'\ ', i)) > n else 1 for i in train["sent"]]
test["more than n space"] = [0 if len(re.findall(r'\ ', i)) > n else 1 for i in test["sent"]]
# Adding more than n question 9
n = 0
train["more than n question"] = [0 if len(re.findall(r'\?', i)) > n else 1 for i in train["sent"]]
test["more than n question"] = [0 if len(re.findall(r'\?', i)) > n else 1 for i in test["sent"]]
# Adding more than n digit 10
n = 1
train["more than n digit"] = [0 if len([num for num in re.findall('\d+', re.sub(r'[^\w\s]', '', sen)) if len(num) > 3]) > n else 1 for sen in train["sent"]]
test["more than n digit"] = [0 if len([num for num in re.findall('\d+', re.sub(r'[^\w\s]', '', sen)) if len(num) > 3]) > n else 1 for sen in test["sent"]]
# Adding more than n money 11
n = 0
train["more than n money"] = [0 if len(re.findall(r"\d*[%$£]+\d*", sen)) > n else 1 for sen in train["sent"]]
test["more than n money"] = [0 if len(re.findall(r"\d*[%$£]+\d*", sen)) > n else 1 for sen in test["sent"]]
# Adding more than n colon 12
n = 0
train["more than n colon"] = [0 if len(re.findall(r'\:', i)) > n else 1 for i in train["sent"]]
test["more than n colon"] = [0 if len(re.findall(r'\:', i)) > n else 1 for i in test["sent"]]

In [9]:
train.head()

,sent,label,more than n punc,more than n words,more than n capital,more than n common,more than n period,more than n equal,more than n (,more than n quotation,more than n space,more than n question,more than n digit,more than n money,more than n colon
0,My children threw a birthday party for me.,positive,1,1,1,1,1,1,1,0,0,1,1,1,1
1,Marketing on social networking sites is a fast...,positive,2,1,1,1,1,1,1,0,0,1,1,1,1
2,"You pillock, look what you've done!",positive,3,1,1,0,1,1,1,0,0,1,1,1,1
3,He scored three touchdowns early in the game.,positive,1,1,1,1,1,1,1,0,0,1,1,1,1
4,His abrupt manner makes me uncomfortable.,positive,1,1,1,1,1,1,1,0,0,1,1,1,1


## Train
Now, it's time to evaluate whether the features just selected is useful to classify. We use [Bernoulli Naive Bayes model](https://scikit-learn.org/stable/modules/naive_bayes.html#bernoulli-naive-bayes) to train training data. 

In [10]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()

<font color="red">**[ TODO ]**</font> Please adjust the `selected_features` list and train the model.

***Don't*** use other model in this assignment.

In [11]:
selected_features = [
                     "more than n punc",
                     "more than n words",
                     "more than n capital",
                     "more than n common",
                     "more than n period",
                     "more than n equal",
                     "more than n (",
                     "more than n quotation",
                     "more than n space",
                     "more than n question",
                     "more than n digit",
                     "more than n money",
                     "more than n colon",
                     ]

print(train[selected_features])

        more than n punc  more than n words  more than n capital  \
0                      1                  1                    1   
1                      2                  1                    1   
2                      3                  1                    1   
3                      1                  1                    1   
4                      1                  1                    1   
...                  ...                ...                  ...   
243943                 7                  1                    0   
243944                11                  0                    0   
243945                 2                  1                    0   
243946                13                  0                    0   
243947                 5                  1                    1   

        more than n common  more than n period  more than n equal  \
0                        1                   1                  1   
1                        1                   

In [12]:
y_pred = bnb.fit(train[selected_features], train['label']).predict(test[selected_features])

## Test
Test the model and get the accuracy of the prediction on testing data. 

There're four baselines for this task:
```
1. simple baseline: 0.65
2. medium baseline: 0.72
3. strong baseline: 0.8
4. boss baseline: 0.85
```
The more the baseline you pass, the more higher the grade you can get.

*hint: If the result isn't ideal, you can print the wrong prediction data and re-observe but only focus on wrong data to extract other features.

In [13]:
print((test['label'] == y_pred).sum() / len(test))

0.8043250327653997


## TA's Notes

If you complete the Assignment, please use [this link](https://docs.google.com/spreadsheets/d/1QGeYl5dsD9sFO9SYg4DIKk-xr-yGjRDOOLKZqCLDv2E/edit#gid=1031097651) to reserve demo time.  
The score is only given after TAs review your implementation, so <u>**make sure you make a appointment with a TA before you miss the deadline**</u> .  <br>After demo, please upload your assignment to eeclass. You just need to hand in this ipynb file and rename it as XXXXXXXXX(Your student ID).ipynb.
<br>Note that **late submission will not be allowed**.